# Introduction
*This notebook is taken from repo [MCQ_prediction](https://github.com/TuringEnterprises/MCQ_predictions/tree/proxy) by Krati Saxena*

One shot Model to predict MCQ percentile. Take developer resume, other MCQ scores, YoE in various things as inputs. 1 Million train examples. We can easily afford 10-100K features. Gultekin Gunduz, Vivek Sehgal

V1:
Simplest features.
Calculated YoE in skill per resume
Self declared YoE in skill in resume
Total Calculated YoE per resume
Total Self declared YoE in resume
Average MCQ percentile in other tech stack tests
Average Seniority score


Possible V2: 
Binary features for each and every MCQ and score in that
University ranking
What is the value of n, assuming this is the n’th MCQ taken.
University names - BoW
Company names - BoW
 Other free text
Country


In [1]:
# Uncomment these lines and replace with your own gcloud credential if needed
#!gcloud auth revoke --all
# !gcloud config set pass_credentials_to_gsutil false
# !gcloud auth login
# !gcloud config set project "turing-230020"
# #!gcloud config set account gultekin.g@turing.com
# !gcloud auth application-default set-quota-project 'turing-230020'
# !gcloud auth application-default login

In [1]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval

def read_data_from_gbq(query, project_id='turing-230020'):
    return pd.io.gbq.read_gbq(query, project_id=project_id)

In [4]:
drop_list = list(set([
    'Work Experience Analysis',
    'English listening',
    'Designer',
    'QA',
    'Reading comprehension',
    'Personality Test',
    'MCQ Survey',
    'Automated Coding Challenge',
    'Golang-Brazil'] + ['Project Collaboration',
    'PHP (New Test)',
    'Work Experience [Advanced]',
    'Work Experience',
    'Work Experience [Basic]',
    'Mobile Development',
    'Algorithm (deprecated)',
    'Automated Seniority Assessment',
    'Web Development (deprecated)',
    'Practice Coding Challenge',
    'Automated Coding Challenge',
    'Product Manager',
    'Project Management',
    'Designer',
    'QA',
    'System Design',
    'Elasticsearch',
    'Team Collaboration',
]))


mcq_ = read_data_from_gbq(

"""

with dev_mcq as (
  select
    dms.dev_id,
    dms.challenge_id,
    dsm.skill_id,
    bas4.skill_name,
    dms.dev_percentile,
    dms.last_updated_at,
    dc.challenge_name
  from 
    external_query("turing-230020.us.machine-learning",
      '''
      select * from prod.dev_mcq_score
      '''
    ) as dms
  left join devdb_mirror.dv2_skill_mcq as dsm
    on dms.challenge_id = dsm.mcq_id
  left join devdb_mirror.base_all_skills_v4 as bas4
    on dsm.skill_id = bas4.id
  left join devdb_mirror.dv2_challenge as dc
    on dms.challenge_id = dc.challenge_id
)

select * from dev_mcq
""")

mcq_.sort_values('last_updated_at', inplace=True)
mcq_ = mcq_[mcq_['challenge_name'].isin(drop_list)==False]
mcq_skill_id_mapping = mcq_.groupby('challenge_id')['skill_id'].unique().to_dict()

D:\Anaconda\envs\turing_ranker_v2\lib\site-packages\google\auth\_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
!pip install pandas_gbq

  Using cached google_cloud_bigquery_storage-2.13.1-py2.py3-none-any.whl (180 kB)
  Using cached pydata_google_auth-1.4.0-py2.py3-none-any.whl (14 kB)
  Using cached google_cloud_bigquery-3.1.0-py2.py3-none-any.whl (211 kB)
  Using cached google_auth_oauthlib-0.5.1-py2.py3-none-any.whl (19 kB)
  Using cached google_auth-2.6.6-py2.py3-none-any.whl (156 kB)
  Using cached db_dtypes-1.0.1-py2.py3-none-any.whl (14 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached protobuf-3.20.1-cp39-cp39-win_amd64.whl (904 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_cloud_core-2.3.0-py2.py3-none-any.whl (29 kB)
  Using cached google_crc32c-1.3.0-cp39-cp39-win_amd64.whl (27 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached idn

In [5]:
dv2_work_experience_avg_score = read_data_from_gbq(
"""
select grade_category_id,
avg_score, 
user_id as dev_id,
submit_time
from devdb_mirror.dv2_work_experience_avg_score d1
left join devdb_mirror.dv2_challenge_submit d2 
on d1.submit_id = d2.submit_id
""")

dv2_work_experience_avg_score.sort_values('submit_time', inplace= True)
dv2_work_experience_avg_score.drop_duplicates(['dev_id','grade_category_id'], keep = 'last', inplace=True)
dv2_work_experience_avg_score

,grade_category_id,avg_score,dev_id,submit_time
597982,3,2.6000,329395,2020-07-12 04:34:19+00:00
301498,2,2.6875,329395,2020-07-12 04:34:19+00:00
1191770,7,2.3750,329395,2020-07-12 04:34:19+00:00
872675,4,1.2500,329395,2020-07-12 04:34:19+00:00
164392,1,3.3000,329395,2020-07-12 04:34:19+00:00
...,...,...,...,...
1311222,7,3.7500,1920597,2022-06-01 23:21:37+00:00
289085,1,4.7000,2238320,2022-06-01 23:21:37+00:00
1073408,4,3.5000,1920597,2022-06-01 23:21:37+00:00
317280,2,4.5625,2238320,2022-06-01 23:21:37+00:00


In [6]:
skill_exp = read_data_from_gbq(
"""
with skill_exp as (
SELECT
  t1.developer_id as dev_id,
  t1.skill_id,
  SUM(t1.project_years_of_experience) AS project_years_of_experience
FROM
  devdb_mirror.tpm_developer_skill_matchability t1
LEFT JOIN 
  devdb_mirror.tpm_developer_skill_matchability t2
ON 
  t1.developer_id = t2.developer_id
  AND t1.skill_id = t2.skill_id
  AND t1.updated_at < t2.updated_at
WHERE
  t2.updated_at IS NULL
GROUP BY 
  1, 2
)
select * from skill_exp 
""")

In [7]:
tpm_developer_skill = read_data_from_gbq(
"""
select developer_id as dev_id,
skill_id,
score, 
skill_level
from devdb_mirror.tpm_developer_skill
"""
)

In [8]:
work_exp = read_data_from_gbq(

"""
select * from devdb_mirror.dv2_work_experience_summary
"""
)

In [9]:
mcq_.sort_values('last_updated_at',inplace=True)
mcq_.drop_duplicates(['dev_id', 'challenge_id'], keep='last', inplace=True)

In [10]:
tpm_dev_grading_result = read_data_from_gbq(
    '''
select * from devdb_mirror.tpm_dev_grading_result
''')

tpm_dev_grading_result.sort_values('updated_date', inplace=True)
temp =tpm_dev_grading_result[['developer_id', 'challenge_score']]
temp.drop_duplicates('developer_id', keep='last', inplace=True)
temp.columns = ['dev_id', 'ti_score']

mcq_ = pd.merge(mcq_, temp, on=['dev_id'], how='left')

C:\Users\H\AppData\Local\Temp\ipykernel_3516\2668128165.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop_duplicates('developer_id', keep='last', inplace=True)


In [11]:
developer_detail = read_data_from_gbq(
'''
select user_id as dev_id,
hourly_rate,
years_of_experience,
calculated_years_of_experience,
github_url,
salary_in_USD, 
country_citizenship_id
from devdb_mirror.developer_detail
''')

In [12]:
visit_list_v4 = read_data_from_gbq(
'''
select uid as dev_id,
       user_os,
       quiz_answer
       from devdb_mirror.visit_list_v4
'''
)
visit_list_v4['has_quiz_answer'] = (visit_list_v4['quiz_answer']!='').astype(int)
visit_list_v4['correct_answer'] = (visit_list_v4['quiz_answer']=='MAX_NUM = NUM').astype(int)
visit_list_v4[visit_list_v4['quiz_answer']=='MAX_NUM = NUM']

,dev_id,user_os,quiz_answer,has_quiz_answer,correct_answer
1,1254315,None,MAX_NUM = NUM,1,1
2,1240529,None,MAX_NUM = NUM,1,1
4,1309279,None,MAX_NUM = NUM,1,1
5,1326508,None,MAX_NUM = NUM,1,1
6,1256783,None,MAX_NUM = NUM,1,1
...,...,...,...,...,...
4493143,399790,Windows 8.1,MAX_NUM = NUM,1,1
4493156,382188,Windows 8.1,MAX_NUM = NUM,1,1
4493158,395967,Windows 8.1,MAX_NUM = NUM,1,1
4493164,405888,Windows 8.1,MAX_NUM = NUM,1,1


In [13]:
quiz_answer = visit_list_v4[visit_list_v4['has_quiz_answer']==1].groupby('dev_id')['correct_answer'].mean().reset_index()
vl4 = visit_list_v4[['dev_id']].drop_duplicates()
vl4 = pd.merge(vl4, quiz_answer, on = ['dev_id'], how='left')

vl4['blank_answer'] = (pd.isnull(vl4['correct_answer'])).astype(int)
vl4

mcq_ = pd.merge(mcq_, vl4, on=['dev_id'], how='left')

In [14]:
def create_mcq_features(df,
                       mcq_id,
                       prefix):
    
    
    ret = {}
    
    #Other MCQ percentiles
    ret[prefix + '_dev_remaining_pct_mean'] = df['dev_percentile'].mean()
    ret[prefix +'dev_remaining_pct_max'] = df['dev_percentile'].max()
    ret[prefix +'dev_remaining_pct_min'] = df['dev_percentile'].min()
    ret[prefix +'dev_remaining_pct_std'] = df['dev_percentile'].std()
    ret[prefix +'dev_num_remaining_mcqs'] = df.shape[0]
            
    return ret

In [15]:
mcq_ = mcq_[pd.isnull(mcq_['dev_percentile'])==False]

In [16]:
developer_detail.drop_duplicates('dev_id', inplace=True)
mcq_ = pd.merge(mcq_, developer_detail, on =['dev_id'], how='left')

In [17]:
mcq_skill_id_mapping.pop(246, None)
mcq_skill_id_mapping

{115: <IntegerArray>
 [268]
 Length: 1, dtype: Int64,
 125: <IntegerArray>
 [165, 2087, 939, 2049]
 Length: 4, dtype: Int64,
 127: <IntegerArray>
 [258]
 Length: 1, dtype: Int64,
 128: <IntegerArray>
 [86, 113, 1186, 2049, 114, 1946, 1821, 2061, 1185]
 Length: 9, dtype: Int64,
 129: <IntegerArray>
 [167]
 Length: 1, dtype: Int64,
 130: <IntegerArray>
 [19, 30]
 Length: 2, dtype: Int64,
 131: <IntegerArray>
 [29]
 Length: 1, dtype: Int64,
 132: <IntegerArray>
 [26, 2096]
 Length: 2, dtype: Int64,
 135: <IntegerArray>
 [2067, 105, 16, 2099, 1625]
 Length: 5, dtype: Int64,
 138: <IntegerArray>
 [160]
 Length: 1, dtype: Int64,
 141: <IntegerArray>
 [567, 1830, 2068, 277, 568, 448, 97, 1909, 108]
 Length: 9, dtype: Int64,
 142: <IntegerArray>
 [54]
 Length: 1, dtype: Int64,
 145: <IntegerArray>
 [2091, 2094]
 Length: 2, dtype: Int64,
 146: <IntegerArray>
 [104]
 Length: 1, dtype: Int64,
 152: <IntegerArray>
 [158]
 Length: 1, dtype: Int64,
 154: <IntegerArray>
 [159]
 Length: 1, dtype: Int6

In [18]:
mcq_ = mcq_[mcq_['challenge_id'].isin(list(mcq_skill_id_mapping.keys()))]

In [35]:
# for _, d in mcq_.groupby(['dev_id']):
# #     print(d
#      for mcq_itr in range(d.shape[0]):
#         print(mcq_itr)

In [ ]:
results = []
import time
from tqdm import tqdm

dev_ids = mcq_["dev_id"].unique().tolist()
with tqdm(total = len(dev_ids)) as pbar:
    for dev_id in dev_ids:
        d = mcq_[mcq_["dev_id"] == dev_id]

        dev_seniority = dv2_work_experience_avg_score[dv2_work_experience_avg_score['dev_id']==dev_id]
        dev_overall_yoe = work_exp[work_exp['developer_id']==dev_id]
        dev_skill_exp = skill_exp[skill_exp['dev_id']==dev_id]
        dev_tpm_developer_skill = tpm_developer_skill[tpm_developer_skill['dev_id']==dev_id]


        r = {}
        r['dev_id'] = dev_id

        for mcq_itr in range(d.shape[0]):

            r['mcq_rank'] = mcq_itr
            dev_percentile = d['dev_percentile'].values[mcq_itr]
            mcq_id = d['challenge_id'].values[mcq_itr]

            r['mcq_id'] = mcq_id
            r['mcq_percentile'] = dev_percentile

            dev_previous_mcqs = d.iloc[:mcq_itr]
            dev_other_mcqs = d[d['challenge_id']!=mcq_id]


            skill_ids = mcq_skill_id_mapping[mcq_id].tolist()

            dev_skill_yoe = dev_skill_exp[dev_skill_exp['skill_id'].isin(skill_ids)]

            dev_skill_selfDeclared_yoe = dev_tpm_developer_skill[dev_tpm_developer_skill['skill_id'].isin(skill_ids)]

            r['dev_skill_yoe'] = dev_skill_yoe['project_years_of_experience'].max()
            r['dev_selfDeclared_skill_yoe'] = dev_skill_selfDeclared_yoe['score'].max()
            r['dev_selfDeclared_skill_level'] = dev_skill_selfDeclared_yoe['skill_level'].max()

            if mcq_itr == 0 :

                for c in ['ti_score', 'correct_answer', 'blank_answer'] + ['hourly_rate', 'years_of_experience', 'calculated_years_of_experience', 'salary_in_USD', 'country_citizenship_id']:

                    r[c] = d[c].values[0]

                r['dev_selfDeclared_overall_YoE'] = dev_overall_yoe['year_experience'].max()
                r['dev_selfDeclared_overall_seniority_id'] = dev_overall_yoe['seniority_id'].max()

                for _ , k in dev_seniority.iterrows():

                    r['seniority_%d' % k['grade_category_id']] = k['avg_score']

                if dev_seniority.shape[0]==5:

                    r['dev_all5_seniority_mean'] = dev_seniority['avg_score'].mean()
                    r['dev_all5_seniority_max'] = dev_seniority['avg_score'].max()
                    r['dev_all5_seniority_min'] = dev_seniority['avg_score'].min()

                r['dev_seniority_mean'] = dev_seniority['avg_score'].mean()
                r['dev_seniority_max'] = dev_seniority['avg_score'].max()
                r['dev_seniority_min'] = dev_seniority['avg_score'].min()

            r.update(create_mcq_features(dev_previous_mcqs,
                        mcq_id,
                       prefix='earlier'))

            results.append(r.copy())
        pbar.update(1)

 41%|█████████████████████████████                                          | 95449/232751 [2:14:03<3:35:40, 10.61it/s]

In [ ]:
df = pd.DataFrame(results)

df['salary_in_USD'] = df['salary_in_USD'].astype(float)

In [ ]:
time_series_features = [
    'mcq_rank',
    'dev_skill_yoe',
    'dev_selfDeclared_skill_yoe',
    'dev_selfDeclared_skill_level',
    'dev_selfDeclared_overall_seniority_id',
    'seniority_1',
    'seniority_2',
    'seniority_3',
    'seniority_4',
    'seniority_7',
    'dev_all5_seniority_mean',
    'dev_all5_seniority_max',
    'dev_all5_seniority_min',
    'dev_seniority_mean',
    'dev_seniority_min',
    'dev_seniority_max',
    'earlier_dev_remaining_pct_mean',
    'earlierdev_remaining_pct_max',
    'earlierdev_remaining_pct_min',
    'earlierdev_remaining_pct_std',   
    'earlierdev_num_remaining_mcqs',
    'ti_score', 
    'correct_answer', 
    'blank_answer',
    'hourly_rate',
    'years_of_experience',
    'calculated_years_of_experience',
    'salary_in_USD',
    'country_citizenship_id',
]

In [ ]:
df[['dev_skill_yoe',
       'dev_selfDeclared_skill_yoe', 'dev_selfDeclared_skill_level',]]

In [ ]:
df[['seniority_3', 'seniority_4', 'seniority_2', 'seniority_7',
       'seniority_1']]

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold, StratifiedKFold

!pip install lightgbm==3.3.1
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")


selecteds = []
bestscore = 9999999

improved = True
report = []
while improved:
    improved = False

    for feat in time_series_features:

        if feat in selecteds:
            continue

        temp_features = [feat] + selecteds

        train_label_col = 'mcq_percentile'

        hyper_params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression_l2',
            'learning_rate': 0.05,
            'feature_fraction': 1.0,
            'bagging_fraction': 1.0,
            'bagging_freq': 1,
            'verbose': -1,
            "max_depth": 8,
            "num_leaves": 31,  
            "max_bin": 200,
            "num_iterations": 100000
        }

        
        
        gkf = GroupKFold(n_splits=4)
        
        yall = np.zeros(len(df))
        for tr, val in gkf.split(df, groups=df['dev_id']):

            train_set = df.iloc[tr]
            val_set = df.iloc[val]

            if 'country_citizenship_id' in temp_features:

                train_data = lgb.Dataset(data=train_set[temp_features],
                                        label=train_set[train_label_col],
                                         categorical_feature=['country_citizenship_id'],
                                        )

                val_data = lgb.Dataset(data=val_set[temp_features],
                                        label=val_set[train_label_col],
                                         categorical_feature=['country_citizenship_id'],
                                        )

                model = lgb.train(train_set=train_data,
                                        params=hyper_params,
                                        num_boost_round=10000,
                                        verbose_eval=False,
                                        early_stopping_rounds=50,
                                        valid_sets=[train_data, val_data],
                                        categorical_feature=['country_citizenship_id']
                                        )

            else:

                train_data = lgb.Dataset(data=train_set[temp_features],
                                        label=train_set[train_label_col],
                                        )

                val_data = lgb.Dataset(data=val_set[temp_features],
                                        label=val_set[train_label_col],
                                        )

                model = lgb.train(train_set=train_data,
                                        params=hyper_params,
                                        num_boost_round=10000,
                                        verbose_eval=False,
                                        early_stopping_rounds=50,
                                        valid_sets=[train_data, val_data]
                                        )

            yall[val]  = model.predict(val_set[temp_features])

            
        curcorr = stats.spearmanr(df[train_label_col].values, yall)[0]
        curscore = mean_squared_error(df[train_label_col].values, yall)
        
        if curscore < bestscore:
            
            bestscore = curscore
            bestcorr = curcorr
            improved = True
            bestfeats = temp_features.copy()
            print([temp_features, curcorr, r2_score(df[train_label_col].values, yall)])
            
    if improved:
        
        selecteds = bestfeats
        
        r = {}
        r['# feats'] = len(selecteds)
        r['used_features'] = selecteds
        r['best_RMSE'] = bestscore
        r['best_Spearman'] = bestcorr
        report.append(r.copy())

In [ ]:
selected_features = ['ti_score', 'hourly_rate', 'salary_in_USD', 'dev_selfDeclared_skill_level', 'earlier_dev_remaining_pct_mean', 'dev_skill_yoe']
selected_features = ['years_of_experience', 'seniority_4', 'seniority_3', 'dev_selfDeclared_skill_yoe', 
                     'calculated_years_of_experience', 
                     'country_citizenship_id', 'seniority_1', 'salary_in_USD', 
                     'dev_selfDeclared_skill_level', 'dev_skill_yoe']
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold, StratifiedKFold
import lightgbm as lgb
import joblib
import warnings
warnings.filterwarnings("ignore")

train_label_col = 'mcq_percentile'

hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'learning_rate': 0.05,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 1,
    'verbose': -1,
    "max_depth": 8,
    "num_leaves": 31,  
    "max_bin": 200,
    "num_iterations": 100000
}



gkf = GroupKFold(n_splits=4)
yall = np.zeros(len(df))

for n , (tr, val) in enumerate(gkf.split(df, groups=df['dev_id'])):

    train_set = df.iloc[tr]
    val_set = df.iloc[val]

    if 'country_citizenship_id' in temp_features:

        train_data = lgb.Dataset(data=train_set[selected_features],
                                label=train_set[train_label_col],
                                 categorical_feature=['country_citizenship_id'],
                                )

        val_data = lgb.Dataset(data=val_set[selected_features],
                                label=val_set[train_label_col],
                                 categorical_feature=['country_citizenship_id'],
                                )

        model = lgb.train(train_set=train_data,
                                params=hyper_params,
                                num_boost_round=10000,
                                verbose_eval=False,
                                early_stopping_rounds=50,
                                valid_sets=[train_data, val_data],
                                categorical_feature=['country_citizenship_id']
                                )

    else:

        train_data = lgb.Dataset(data=train_set[selected_features],
                                label=train_set[train_label_col],
                                )

        val_data = lgb.Dataset(data=val_set[selected_features],
                                label=val_set[train_label_col],
                                )

        model = lgb.train(train_set=train_data,
                                params=hyper_params,
                                num_boost_round=10000,
                                verbose_eval=False,
                                early_stopping_rounds=50,
                                valid_sets=[train_data, val_data]
                                )

        
    # save model
    joblib.dump(model, 'mcq_prediction_model_fold%d.pkl' % n)
    
    yall[val]  = model.predict(val_set[selected_features])


curcorr = stats.spearmanr(df[train_label_col].values, yall)[0]
curscore = mean_squared_error(df[train_label_col].values, yall)
print([temp_features, curcorr, r2_score(df[train_label_col].values, yall)])